In [76]:
# import torch
# import torch.nn as nn
# from torch.optim import Adam
# from torch.utils.data import random_split
# import numpy as np
# from sklearn.metrics import accuracy_score
# from scipy.stats import entropy
import regex as re
from tqdm import tqdm
import pandas as pd
import threading
import multiprocessing as mp
import time
from IPython.display import clear_output

try:
    del users
except:
    pass

In [77]:
# while True:
#     users = pd.read_csv("user_reviews_multithread.csv", on_bad_lines='skip')
#     users = users.drop_duplicates()
#     df_shape = users.shape
#     df_duplicated = users.duplicated().any()
#     df_users_unique  = users.user.nunique()
#     df_movies_unique = users.movie.nunique()
#     currently_scraping = users.groupby("movie").size()[users.groupby("movie").size() < 720]
#     clear_output(wait=True)
#     print("Dataframe shape:", df_shape)
#     print("Duplicates:", df_duplicated)
#     print("Unique users:", df_users_unique)
#     print("Unique movies: ", df_movies_unique)
#     print()
#     print(f"Pages completed: {df_movies_unique / 72:.2f}")
#     print()
#     print("Currently scraping:")
#     print(currently_scraping)

#     del users

#     time.sleep(3)
# users.tail()

In [78]:
reviews = pd.read_csv("user_reviews_multithread.csv", on_bad_lines='skip')

In [79]:
print("Unique users: ", len(reviews.user.unique()))
print("Unique movies: ", len(reviews.movie.unique()))
print("Total reviews: ", len(reviews))
reviews.head(3)

Unique users:  102023
Unique movies:  2504
Total reviews:  1802880


,film_popularity,review_popularity,rating,liked,user,date,comments,movie,review,likes
0,1,1,10,1,b1mb1m,15 Jul 2023,199,barbie,s(he's) bro(ken),95535
1,1,2,10,1,aksually,21 Jul 2023,116,barbie,can someone please check on Allan im afraid he...,67991
2,1,3,8,0,timozinho,24 Jul 2023,83,barbie,Dropping the kids off at Oppenheimer so the ad...,60920


In [80]:
reviews = reviews.sort_values(by=["film_popularity", "review_popularity"], ascending=[True, True])

In [81]:
movies = pd.read_csv("letterboxd_saved.csv")

In [82]:
# clean strings from five stars column
movies['five stars'] = pd.to_numeric(movies['five stars'], errors='coerce')
movies = movies.dropna(subset=['five stars'])

# add column for average rating
weights = list(range(1, 11))
weights = [w / 2 for w in weights]
movies["avg rating"] = movies.iloc[:, 6:16].dot(weights) / movies.iloc[:, 6:16].sum(axis=1)

In [86]:
# strip code title from url
movies["url_movie_name"] = movies["url"].astype(str).apply(lambda x: x.rstrip("/").split("/")[-1] if "/" in x else None)
years = movies[["movie", "url_movie_name", "year", "avg rating"]]
# years = years.rename(columns={"movie": "title", "url_movie_name": "movie"})

# add movie release year to reviews dataframe
mapping = years.set_index("url_movie_name")["year"].to_dict()
reviews["release_year"] = reviews["movie"].map(mapping)

# add average rating to reviews dataframe
mapping = years.set_index("url_movie_name")["avg rating"].to_dict()
reviews["avg rating"] = reviews["movie"].map(mapping)

# reviews["release_year"] = reviews["release_year"].astype(int)
# reviews["avg rating"] = reviews["avg rating"].astype(float)

In [87]:
reviews[reviews.release_year.isna()].movie.nunique()

136

In [88]:
reviews.release_year.isna().any()

True

In [89]:
reviews.head()
# reviews.to_csv("reviews_cleaned.csv", index=None)

,film_popularity,review_popularity,rating,liked,user,date,comments,movie,review,likes,release_year,avg rating
0,1,1,10,1,b1mb1m,15 Jul 2023,199,barbie,s(he's) bro(ken),95535,2023.0,3.863179
1,1,2,10,1,aksually,21 Jul 2023,116,barbie,can someone please check on Allan im afraid he...,67991,2023.0,3.863179
2,1,3,8,0,timozinho,24 Jul 2023,83,barbie,Dropping the kids off at Oppenheimer so the ad...,60920,2023.0,3.863179
3,1,4,10,1,jeaba,20 Jul 2023,82,barbie,"historically inaccurate film, all of my barbie...",42910,2023.0,3.863179
4,1,5,10,1,zoerosebryant,14 Jul 2023,288,barbie,we all knew barbie was going to be funny. we a...,33202,2023.0,3.863179


In [90]:
movies.head()
# movies.to_csv("movies_cleaned.csv", index=None)

,movie,year,Watched by x members,Appears in x lists,Liked by x members,No in Top 250,half stars,one stars,onehalf stars,two stars,...,tagline,description,cast,crew,details,genres,themes,url,avg rating,url_movie_name
0,Barbie,2023,5112770,519663,2113121,NaN,19336,44937,28705,152802,...,She’s everything. He’s just Ken.,Barbie and Ken are having the time of their li...,"['Margot Robbie', 'Ryan Gosling', 'America Fer...","{'Director': ['Greta Gerwig'], 'Producers': ['...","{'Studios': ['LuckyChap Entertainment', 'Heyda...","['Comedy', 'Adventure']","['Humanity and the world around us', 'Crude hu...",https://letterboxd.com/film/barbie/,3.863179,barbie
1,Parasite,2019,4932332,642601,2693669,9.0,3263,7688,3279,22196,...,Act like you own the place.,"All unemployed, Ki-taek’s family takes peculia...","['Song Kang-ho', 'Lee Sun-kyun', 'Cho Yeo-jeon...","{'Director': ['Bong Joon Ho'], 'Producers': ['...","{'Studio': ['Barunson E&A'], 'Country': ['Sout...","['Comedy', 'Thriller', 'Drama']","['Humanity and the world around us', 'Intense ...",https://letterboxd.com/film/parasite-2019/,4.528420,parasite-2019
2,Interstellar,2014,4956733,613257,2372421,53.0,5444,13846,7071,45823,...,Mankind was born on Earth. It was never meant ...,The adventures of a group of explorers who mak...,"['Matthew McConaughey', 'Anne Hathaway', 'Mich...","{'Director': ['Christopher Nolan'], 'Producers...","{'Studios': ['Legendary Pictures', 'Syncopy', ...","['Science Fiction', 'Drama', 'Adventure']","['Monsters, aliens, sci-fi and the apocalypse'...",https://letterboxd.com/film/interstellar/,4.444973,interstellar
3,Fight Club,1999,4979960,532959,2289970,179.0,4937,13048,6534,43359,...,Mischief. Mayhem. Soap.,A ticking-time-bomb insomniac and a slippery s...,"['Edward Norton', 'Brad Pitt', 'Helena Bonham ...","{'Director': ['David Fincher'], 'Producers': [...","{'Studios': ['Fox 2000 Pictures', 'Regency Ent...",['Drama'],"['Intense violence and sexual transgression', ...",https://letterboxd.com/film/fight-club/,4.303348,fight-club
4,La La Land,2016,4328732,611410,1950863,NaN,14230,43529,15017,104979,...,Here’s to the fools who dream.,"Mia, an aspiring actress, serves lattes to mov...","['Ryan Gosling', 'Emma Stone', 'John Legend', ...","{'Director': ['Damien Chazelle'], 'Producers':...","{'Studios': ['Summit Entertainment', 'Gilbert ...","['Drama', 'Comedy', 'Music', 'Romance']","['Song and dance', 'Humanity and the world aro...",https://letterboxd.com/film/la-la-land/,4.149215,la-la-land
